# Titanic Death Recipy

---

---


# 1. Competition Description
---

The sinking of the RMS Titanic is one of the most infamous shipwrecks in history.  On April 15, 1912, during her maiden voyage, the Titanic sank after colliding with an iceberg, killing 1502 out of 2224 passengers and crew. This sensational tragedy shocked the international community and led to better safety regulations for ships.

One of the reasons that the shipwreck led to such loss of life was that there were not enough lifeboats for the passengers and crew. Although there was some element of luck involved in surviving the sinking, some groups of people were more likely to survive than others, such as women, children, and the upper-class.

In this challenge, we ask you to complete the analysis of what sorts of people were likely to survive. In particular, we ask you to apply the tools of machine learning to predict which passengers survived the tragedy.


## Data structure


| Variable | Definition                                 | Key                                            |
| -------- | ------------------------------------------ | ---------------------------------------------- |
| survival | Survival                                   | 0 = No, 1 = Yes                                |
| pclass   | Ticket class                               | 1 = 1st, 2 = 2nd, 3 = 3rd                      |
| sex      | Sex                                        |                                                |
| Age      | Age in years                               |                                                |
| sibsp    | # of siblings / spouses aboard the Titanic |                                                |
| parch    | # of parents / children aboard the Titanic |                                                |
| ticket   | Ticket number                              |                                                |
| fare     | Passenger fare                             |                                                |
| cabin    | Cabin number                               |                                                |
| embarked | Port of Embarkation                        | C = Cherbourg, Q = Queenstown, S = Southampton |
    


## Variable Notes

**pclass**: A proxy for socio-economic status (SES)
1st = Upper
2nd = Middle
3rd = Lower

**age**: Age is fractional if less than 1. If the age is estimated, is it in the form of xx.5

**sibsp**: The dataset defines family relations in this way...
Sibling = brother, sister, stepbrother, stepsister
Spouse = husband, wife (mistresses and fiancés were ignored)

**parch**: The dataset defines family relations in this way...
Parent = mother, father
Child = daughter, son, stepdaughter, stepson
Some children travelled only with a nanny, therefore parch=0 for them.

# 2. Data analysis
---

We use pandas librery to import the data as a DataFrame and do some quick analysis

In [1]:
import pandas as pd

Dt = pd.read_csv("train.csv",index_col="PassengerId")
Dt.head(5)

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
PassengerId,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## 2.1 Survivors Rate

Here we will analyse the rate of survivors in general and in subsets

In [2]:
# We define the function SurvRate to calculate the desired rate
def SurvRate(Dataset):
    
    SubGroup = Dataset[['Survived']]

    
    NTotal = SubGroup.count()
    NSurv  = SubGroup[(Dataset['Survived'] == 1)].count()
    
    Result            = NSurv/NTotal
    Result['Total']   = NTotal['Survived']
    Result['Surv'] = NSurv['Survived']
    
    return Result.rename({"Survived": "SR", "Surv": "Survived"})


Dt.groupby(['Pclass','Sex','Embarked']).apply(SurvRate)



SR  Total  Survived
Pclass Sex    Embarked                           
1      female C         0.976744   43.0      42.0
              Q         1.000000    1.0       1.0
              S         0.958333   48.0      46.0
       male   C         0.404762   42.0      17.0
              Q         0.000000    1.0       0.0
              S         0.354430   79.0      28.0
2      female C         1.000000    7.0       7.0
              Q         1.000000    2.0       2.0
              S         0.910448   67.0      61.0
       male   C         0.200000   10.0       2.0
              Q         0.000000    1.0       0.0
              S         0.154639   97.0      15.0
3      female C         0.652174   23.0      15.0
              Q         0.727273   33.0      24.0
              S         0.375000   88.0      33.0
       male   C         0.232558   43.0      10.0
              Q         0.076923   39.0       3.0
              S         0.128302  265.0      34.0

## 3. Models
---

In [6]:
import scipy.stats as stats

def SexNum(Row):
    if Row['Sex'] == 'female':
        return 1
    else:
        return 0

SDt = Dt.copy()
SDt['Sexnum'] = Dt.apply(SexNum,axis=1)



,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Sexnum
PassengerId,,,,,,,,,,,,
1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,1
3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,1
5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,0
6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,0
7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,0
8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,0
9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,1
